<a href="https://colab.research.google.com/github/SamNavdeep/24b1303_AIC/blob/main/AIC_question_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
#=== import the libraries====
import torch
import numpy as np
import pandas as pd
import transformers
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')
from transformers import AutoTokenizer,AutoModel
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from transformers import BertTokenizer, BertForSequenceClassification


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [38]:
#====loading the data======
from numpy import array
csv_file_location='/content/train (2).csv'
data=pd.read_csv(csv_file_location)
category=data['Category']
text=data['Text']
category_arr= array(category)
category_names=list(set(category_arr))
category_names.sort()
category_names=np.array(category_names)
print(category_names)

tokenize = AutoTokenizer.from_pretrained("bert-base-uncased")

token=[]
stop_words=list(set((stopwords.words('english'))))
stop_words_2=[]
for words in stop_words:
  upper_case=words.upper()
  stop_words_2.append(upper_case)
all_stop_words = stop_words + stop_words_2
for n in range(len(text)):
  token_array=word_tokenize(text[n])
  token_array=[word for word in token_array if not word in all_stop_words]
  token.append(token_array)

['Accountant' 'Advocate' 'Agriculture' 'Apparel' 'Architecture' 'Arts'
 'Automobile' 'Aviation' 'BPO' 'Banking' 'Blockchain'
 'Building and Construction' 'Business Analyst' 'Civil Engineer'
 'Consultant' 'Data Science' 'Database' 'Designing' 'DevOps'
 'Digital Media' 'DotNet Developer' 'ETL Developer' 'Education'
 'Electrical Engineering' 'Finance' 'Food and Beverages'
 'Health and Fitness' 'Human Resources' 'Information Technology'
 'Java Developer' 'Management' 'Mechanical Engineer'
 'Network Security Engineer' 'Operations Manager' 'PMO' 'Public Relations'
 'Python Developer' 'React Developer' 'SAP Developer' 'SQL Developer'
 'Sales' 'Testing' 'Web Designing']


In [39]:
#===lemmatization========
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer= WordNetLemmatizer()


tags=[]
for n in range(len(token)):
   tags.append(list(nltk.pos_tag(token[n])))

def tagger(tag):
  if tag.startswith('J'):
    return wordnet.ADJ
  elif tag.startswith('V'):
        return wordnet.VERB
  elif tag.startswith('N'):
        return wordnet.NOUN
  elif tag.startswith('R'):
        return wordnet.ADV
  else:
        return None


final_tag=[]
for n in range(len(tags)):
 final_tag.append(list(map(lambda x :(x[0], tagger(x[1])), tags[n])))

lemmatized_array=[]
for n in range(len(final_tag)):
  lemmatized_subarray=[]
  for word, tag in final_tag[n]:
    if tag is None:
      lemmatized_subarray.append(word)
    else:
      lemmatized_subarray.append(lemmatizer.lemmatize(word, tag))
  lemmatized_array.append(lemmatized_subarray)
lemmatized_text=[]
for subarray in lemmatized_array:
  joined_text = " ".join(subarray)
  lemmatized_text.append(joined_text)

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [40]:
#=====dividing the data into test and training sets=========
import random
random.seed(10)
random.shuffle(lemmatized_text)
random.seed(10)
random.shuffle(category_arr)
n=int((len(lemmatized_text))*0.8)
training_data=lemmatized_text[:n]
val_data=lemmatized_text[n:]
training_labels=category_arr[:n]
val_labels=category_arr[n:]

from transformers import AutoTokenizer
tokenizer=AutoTokenizer.from_pretrained('bert-base-uncased')
token_train=tokenizer(training_data, padding='max_length', max_length=512, truncation=True)
token_val=tokenizer(val_data, padding='max_length', max_length=512, truncation=True)

train_id = torch.tensor(token_train['input_ids'])
train_mask = torch.tensor(token_train['attention_mask'])
val_id = torch.tensor(token_val['input_ids'])
val_mask = torch.tensor(token_val['attention_mask'])

from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
train_labels = torch.tensor(encoder.fit_transform(training_labels))
val_labels = torch.tensor(encoder.fit_transform(val_labels))



In [41]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size=32
train_data = TensorDataset(train_id, train_mask, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
val_data = TensorDataset(val_id, val_mask, val_labels)
val_sampler = SequentialSampler(val_data)
val_dataloader = DataLoader(val_data, sampler=val_sampler, batch_size=batch_size)

model_0= BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=43)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [35]:
# #============freezing========
# for param in model_0.parameters():
#   param.requires_grad=False


In [ ]:
#============model_0 training============
from torch.optim import AdamW
import torch.nn as nn
optimizer = AdamW(model_0.parameters(),lr = 0.0001)

cross_entropy  = nn.NLLLoss()
train_acc, val_acc = [], []
num_epochs = 5
def evaluate(model,loader):
    model.eval()
    total, correct = 0, 0
    with torch.no_grad():
       for steps, batch in enumerate(loader):
          input_ids = batch[0]
          attention_mask = batch[1]
          labels = batch[2]

          outputs = model_0(
          input_ids=input_ids,
          attention_mask=attention_mask,
          labels=labels)
          logits=outputs.logits
          _, preds = torch.max(logits, 1)
          total += batch.size(0)
          correct += (preds == labels).sum().item()
    return correct / total


for epoch in range(num_epochs):
    model_0.train()
    for steps,batch in enumerate(train_dataloader):
        input_ids = batch[0]
        attention_mask = batch[1]
        labels = batch[2]

        outputs = model_0(
        input_ids=input_ids,
        attention_mask=attention_mask,
        labels=labels)
        optimizer.zero_grad()
        loss = outputs.loss
        logits=outputs.logits
        loss.backward()
        optimizer.step()

train_acc = evaluate(model_0, train_dataloader)
val_acc = evaluate(model_0, val_dataloader)
train_acc.append(train_acc)
val_acc.append(val_acc)

In [ ]:
#==========fine tuning==============
for param in model_0.parameters():
    param.requires_grad = True

for epoch in range(3):
    model_0.train()
    for steps,batch in enumerate(train_dataloader):
        outputs = model_0(
        input_ids=batch['input_ids'],
        attention_mask=batch['attention_mask'],
        labels=batch['labels'])
        optimizer.zero_grad()
        loss = outputs.loss
        logits=outputs.logits
        loss.backward()
        optimizer.step()
    val_acc=evaluate(model_0, val_dataloader)

